In [3]:
# add source directory to path
import sys
sys.path.append('../')

import src.models.model_buildings as model_buildings
import src.models.model_hvac as model_hvac
import src.simulator as simulator
from src.disturbances import generate_disturbances
from data.buildings.sfh_1919_1948 import sfh_1919_1948_0_soc as building

# Initialize the building model
bldg_model = model_buildings.Building(params    = building, # More example buildings can be found in data/buildings/.
                                      mdot_hp   = 0.25,          # Massflow of the heat supply system. [kg/s]
                                      method    = '2R2C',        # For available calculation methods see: model_buildings.py.
                                      )

# Initalize the heat pump model
hp_model = model_hvac.Heatpump_AW(mdot_HP = 0.25)

# Generate disturbances
p_hourly = generate_disturbances(bldg_model)[0:168]

# define timestep
timestep = 3600 #s
p = p_hourly.resample(f'{timestep}S').pad() # resample disturbance

# setup simulator
Simulator = simulator.Model_simulator(bldg_model = bldg_model,
                                      hp_model   = hp_model,
                                      timestep   = timestep)

# Set the control variable
uk = 20  # T_hp_sup [°C]

# Get available state keys from building_model
state_keys = bldg_model.state_keys

# Setup initial state vector
x_init = {key : 20 for key in state_keys} # here all inital states are set to 20 °C

48.0252 7.7184 
Save PVGIS data to file data/weather/pvgis_480252_077184_2015_2015.json


FileNotFoundError: [Errno 2] No such file or directory: 'data\\weather\\pvgis_480252_077184_2015_2015.json'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
from controller.heatcurve import Heatingcurve
from report import plot_bldg_temperatures
plt.rcParams["figure.figsize"] = (14,2)

# create instance of heat curve
HC = Heatingcurve()

def draw_heatcurve(shift, offset):
    # Heat curve
    T_amb = np.linspace(-20,35,56)
    T_sup, T_ret = HC.calc(T_amb, shift, offset)
    T_sup_nom, T_ret_nom = HC.calc(T_amb)
    plt.plot(T_amb, T_sup, label = "T_sup [°C]")
    plt.plot(T_amb, T_sup_nom, label = "T_sup_nom [°C]")
    plt.ylim(0,100)
    plt.legend()
    plt.show()

    # Heatingcurve controller
    results = Simulator.simulate(x_init=x_init, T_room_set=20, p=p, ctrl_method='heatcurve', shift = shift, offset = offset)
    
    plt.plot(results['states'], label = list(results['states'].columns + ' [°C]'))
    plt.plot(results['control'], label = 'T_sup [°C]')
    plt.plot(p_hourly['T_amb'], label = 'T_amb [°C]')
    plt.legend()
    plt.show()

interact_manual(draw_heatcurve, shift = (-10.0,10.0), offset = (-20.0,50.0), continuous_update = False)